In [2]:
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

def calculate_iou(boxA, boxB):
    """计算两个边界框的IoU"""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    box_a_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    box_b_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    return inter_area / float(box_a_area + box_b_area - inter_area)

def parse_gt_boxes(label_path, img_size):
    """解析标注文件中的第0类边界框"""
    gt_boxes = []
    try:
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                if int(parts[0]) != 0:
                    continue
                    
                x_center = float(parts[1]) * img_size[0]
                y_center = float(parts[2]) * img_size[1]
                width = float(parts[3]) * img_size[0]
                height = float(parts[4]) * img_size[1]
                
                x1 = x_center - width / 2
                y1 = y_center - height / 2
                x2 = x_center + width / 2
                y2 = y_center + height / 2
                
                gt_boxes.append([x1, y1, x2, y2])
    except:
        pass
    return gt_boxes

def evaluate_model(model_path, image_root, label_root, iou_threshold=0.5, conf_threshold=0.5):
    """评估模型在所有子目录中的第0类数据"""
    model = YOLO(model_path)
    model.conf = conf_threshold
    
    total_gt = 0
    total_tp = 0
    total_fp = 0
    total_images = 0
    
    # 遍历所有子目录（如train/val/test）
    for subdir in os.listdir(image_root):
        image_subdir = os.path.join(image_root, subdir)
        if not os.path.isdir(image_subdir):
            continue
        
        label_subdir = os.path.join(label_root, subdir)
        if not os.path.isdir(label_subdir):
            continue
        
        # 遍历子目录中的图像文件
        for img_name in os.listdir(image_subdir):
            if not img_name.lower().endswith(('.png', '.jpg')):
                continue
            
            img_path = os.path.join(image_subdir, img_name)
            label_name = os.path.splitext(img_name)[0] + '.txt'
            label_path = os.path.join(label_subdir, label_name)
            
            # 筛选属于第0类的图像
            has_class0 = False
            try:
                with open(label_path, 'r') as f:
                    for line in f:
                        if line.strip().split()[0] == '0':
                            has_class0 = True
                            break
            except:
                continue
            
            if not has_class0:
                continue
            
            # 获取图像尺寸
            try:
                img = Image.open(img_path)
                img_size = (img.width, img.height)
            except:
                continue
            
            # 获取标注框
            gt_boxes = parse_gt_boxes(label_path, img_size)
            if not gt_boxes:
                continue
            
            # 模型推理
            results = model.predict(img_path, save=False)
            pred_boxes = []
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    if int(box.cls) == 0:
                        xyxy = box.xyxy[0].tolist()
                        pred_boxes.append(xyxy)
            
            # 计算指标
            matched_gt = [False] * len(gt_boxes)
            for pred in pred_boxes:
                max_iou = 0
                best_gt_idx = -1
                for i, gt in enumerate(gt_boxes):
                    iou = calculate_iou(pred, gt)
                    if iou > max_iou:
                        max_iou = iou
                        best_gt_idx = i
                if max_iou >= iou_threshold and not matched_gt[best_gt_idx]:
                    matched_gt[best_gt_idx] = True
            
            # 统计
            tp = sum(matched_gt)
            fp = len(pred_boxes) - tp
            fn = len(gt_boxes) - tp
            
            total_tp += tp
            total_fp += fp
            total_gt += len(gt_boxes)
            total_images +=1
    
    precision = total_tp / (total_tp + total_fp + 1e-6)
    recall = total_tp / (total_gt + 1e-6)
    
    return {
        'precision': precision,
        'recall': recall,
        'total_images': total_images,
        'total_gt': total_gt
    }

if __name__ == "__main__":
    model_paths = [
        "/data/lizhi/yolo/yolo_train_results/yolo11n/tooth_defect_11n/weights/best.pt",
        "/data/lizhi/yolo/yolo_train_results/yolo12n/tooth_defect_12n/weights/best.pt",
        "/data/lizhi/yolo/yolo_train_results/yolov8n/tooth_defect_v8n/weights/best.pt",
        "/data/lizhi/yolo/yolo_train_results/yolov10n/tooth_defect_v10n/weights/best.pt"
    ]
    
    image_root = "/data/lizhi/yolo/dataset/dataset/images"
    label_root = "/data/lizhi/yolo/dataset/dataset/labels"
    
    results = {}
    for model_path in model_paths:
        model_name = os.path.basename(os.path.dirname(model_path))
        print(f"Evaluating {model_name}...")
        metrics = evaluate_model(model_path, image_root, label_root)
        results[model_name] = metrics
    
    # 输出结果
    print("\nEvaluation Results:")
    for model_name, metric in results.items():
        print(f"Model: {model_name}")
        print(f"  Precision: {metric['precision']:.4f}")
        print(f"  Recall: {metric['recall']:.4f}")
        print(f"  Total Images Evaluated: {metric['total_images']}")
        print(f"  Total Ground Truth Boxes: {metric['total_gt']}\n")

Evaluating weights...

image 1/1 /data/lizhi/yolo/dataset/dataset/images/val/27_slice_0097.png: 640x640 1 tooth_defect, 5.0ms
Speed: 54.9ms preprocess, 5.0ms inference, 646.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /data/lizhi/yolo/dataset/dataset/images/val/23_slice_0274.png: 640x640 1 tooth_defect, 7.4ms
Speed: 3.2ms preprocess, 7.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /data/lizhi/yolo/dataset/dataset/images/val/1_slice_0122.png: 640x640 (no detections), 5.0ms
Speed: 2.6ms preprocess, 5.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /data/lizhi/yolo/dataset/dataset/images/val/20_slice_0111.png: 640x640 1 tooth_defect, 5.4ms
Speed: 2.8ms preprocess, 5.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /data/lizhi/yolo/dataset/dataset/images/val/19_slice_0106.png: 640x640 1 tooth_defect, 5.0ms
Speed: 2.5ms preprocess, 5.0ms inference, 2.2ms postprocess per image a

In [32]:
import os
import shutil

def is_folder_empty(folder_path):
    return not any(os.scandir(folder_path))

def main():
    # 配置路径
    base_models_dir = "/data/lizhi/yolo/yolo_train_results"
    target_dir = "/data/lizhi/yolo/yolo_train_vis_"
    
    # 手动指定的文件列表
    # selected_files = [
    #     "Kavo_47.nii_slice150.jpg",
    #     "Meyer_24.nii_slice095.jpg",
    #     "Meyer_17.nii_slice116.jpg",
    #     "Meyer_54.nii_slice144.jpg",
    #     "Meyer_68.nii_slice204.jpg",
    #     "Newtown_14.nii_slice431.jpg"
    # ]
    
    selected_files = [
        "Kavo_47.nii_slice153.jpg",
        "Meyer_103.nii_slice192.jpg",
        "Newtown_14.nii_slice233.jpg",
    ]

    # 获取所有模型名称（排除空文件夹）
    model_names = []
    for name in os.listdir(base_models_dir):
        model_path = os.path.join(base_models_dir, name)
        if os.path.isdir(model_path) and not is_folder_empty(model_path):
            model_names.append(name)
    
    # 创建目标目录并复制文件
    for model_name in model_names:
        model_target_dir = os.path.join(target_dir, model_name)
        os.makedirs(model_target_dir, exist_ok=True)
        
        model_vis_dir = os.path.join(base_models_dir, model_name, "test_results/visualization")
        
        for file in selected_files:
            src_path = os.path.join(model_vis_dir, file)
            dst_path = os.path.join(model_target_dir, file)
            if os.path.exists(src_path):
                shutil.copy(src_path, dst_path)
            else:
                print(f"Warning: File {src_path} not found for model {model_name}")
    
    print(f"Selected {len(selected_files)} files:")
    for f in selected_files:
        print(f"  {f}")

if __name__ == "__main__":
    main()

Selected 3 files:
  Kavo_47.nii_slice153.jpg
  Meyer_103.nii_slice192.jpg
  Newtown_14.nii_slice233.jpg


In [ ]:
from ultralytics import YOLO

# 加载分类模型配置（如YOLOv11分类专用配置）
model = YOLO('yolov11n-cls.yaml')  # 使用分类任务的模型配置文件

# 训练模型
results = model.train(
    data='/data/lizhi/yolo/dataset/dataset/dataset.yaml',          # 数据配置文件路径
    epochs=100,                   # 训练轮数
    imgsz=224,                    # 输入图像尺寸（如224x224）
    batch=16,                     # 批次大小（根据显存调整）
    device='0',                   # 使用GPU（如'0'表示第一个GPU）
    project='runs/cls',           # 结果保存路径
    name='exp1'                   # 实验名称
)

In [5]:
from ultralytics import YOLO
from pathlib import Path

def train_yolo(
    data_yaml: str,
    model_type: str = "yolov8n.pt",
    output_dir: str = "/data/lizhi/yolo/yolo_train_results",
    epochs: int = 300,
    imgsz: int = 640,
    batch: int = 16,
    device: str = "0",
    name: str = "tooth_defect_nano"
):
    # 创建输出目录
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # 加载模型
    model = YOLO(model_type)
    
    # 训练模型
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=name,
        project=output_dir,
        device=device,
        workers=8,
        patience=50,
        save_period=20,
        exist_ok=True,
        augment=True,
        flipud=0.5,
        fliplr=0.5,
        mosaic=True
    )
    
    # 直接返回保存目录路径（无需索引）
    save_dir = Path(output_dir) / "detect" / name
    print(f"Training results saved to: {save_dir}")
    return str(save_dir)

if __name__ == "__main__":
    data_yaml_path = "/data/lizhi/yolo/yolo_dataset/dataset.yaml"
    output_dir = "/data/lizhi/yolo/yolo_train_results_yolo11"
    train_yolo(
        data_yaml=data_yaml_path,
        model_type="yolov8n.pt",
        output_dir=output_dir,
        epochs=300,
        imgsz=640,
        batch=16,
        name="tooth_defect_nano"
    )

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/data/lizhi/yolo/yolo_dataset/dataset.yaml, epochs=300, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=20, cache=False, device=0, workers=8, project=/data/lizhi/yolo/yolo_train_results, name=tooth_defect_nano, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning /data/lizhi/yolo/yolo_dataset/labels/train.cache... 2830 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2830/2830 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice171.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice172.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice173.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice174.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice175.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice176.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice177.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/Newtown_11.nii_slice178.png: 1 duplicate labels removed
train: WARNING ⚠️ /data/lizhi/yolo/yolo_dataset/images/train/New


val: Scanning /data/lizhi/yolo/yolo_dataset/labels/val.cache... 675 images, 0 backgrounds, 0 corrupt: 100%|██████████| 675/675 [00:00<?, ?it/s]


Plotting labels to /data/lizhi/yolo/yolo_train_results/tooth_defect_nano/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /data/lizhi/yolo/yolo_train_results/tooth_defect_nano
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    150/300      2.81G     0.6436     0.3698     0.8686         20        640: 100%|██████████| 177/177 [00:23<00:00,  7.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:07<00:00,  2.94it/s]

                   all        675        856      0.611      0.396      0.488      0.176
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 100, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



150 epochs completed in 0.689 hours.
Optimizer stripped from /data/lizhi/yolo/yolo_train_results/tooth_defect_nano/weights/last.pt, 6.3MB
Optimizer stripped from /data/lizhi/yolo/yolo_train_results/tooth_defect_nano/weights/best.pt, 6.3MB

Validating /data/lizhi/yolo/yolo_train_results/tooth_defect_nano/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:06<00:00,  3.31it/s]


                   all        675        856      0.643      0.403      0.483      0.189
Speed: 0.1ms preprocess, 3.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /data/lizhi/yolo/yolo_train_results/tooth_defect_nano
Training results saved to: /data/lizhi/yolo/yolo_train_results/detect/tooth_defect_nano


In [19]:
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt

def validate_yolo(
    model_path: str,
    data_yaml: str,
    test_images_dir: str,
    output_dir: str = "/data/lizhi/yolo/validation_results",
    batch_size: int = 64,
    device: str = "0",
    conf_thres: float = 0.2,
    iou_thres: float = 0.6
):
    # 创建输出目录
    save_dir = Path(output_dir)
    save_dir.mkdir(parents=True, exist_ok=True)

    # 加载训练好的最佳模型
    model = YOLO(model_path)

    # 第一阶段：指标验证
    val_metrics = model.val(
        data=data_yaml,
        split='test',  # 确保dataset.yaml中test路径已正确配置
        batch=batch_size,
        device=device,
        conf=conf_thres,
        iou=iou_thres,
        save_json=True,    # 保存JSON格式结果
        save_hybrid=True,  # 保存混合标签结果
        plots=True,       # 生成所有评估图表
        project=output_dir,
        name='metrics'
    )

    # 第二阶段：可视化预测结果
    predictions = model.predict(
        source=test_images_dir,
        conf=conf_thres,
        save=True,          # 保存带预测框的图片
        save_txt=True,      # 保存标签文件
        save_conf=True,    # 保存置信度分数
        save_crop=True,     # 保存裁剪的检测目标
        show_labels=True,
        show_conf=True,
        show_boxes=True,
        project=output_dir,
        name='visualization',
        exist_ok=True
    )

    # 第三阶段：生成附加可视化
    # 1. 绘制混淆矩阵
    (save_dir / 'metrics' / 'confusion_matrix.png').exists() and \
        plt.imshow(plt.imread(str(save_dir / 'metrics' / 'confusion_matrix.png')))
    plt.title('Confusion Matrix')
    plt.axis('off')
    plt.savefig(str(save_dir / 'confusion_matrix_visualized.png'), bbox_inches='tight')
    plt.close()

    return str(save_dir)

if __name__ == "__main__":
    validate_yolo(
        model_path="/data/lizhi/yolo/yolo_train_results_yolo11/tooth_defect_nano_/weights/best.pt",
        data_yaml="/data/lizhi/yolo/yolo_dataset/dataset.yaml",
        test_images_dir="/data/lizhi/yolo/yolo_dataset/images/test",
        output_dir="/data/lizhi/yolo/validation_results_yolo11_test",
        batch_size=64,
        device="0",
        conf_thres = 0.26,
        iou_thres = 0.6
    )

WARNING ⚠️ 'save_hybrid=True' will append ground truth to predictions for autolabelling.
WARNING ⚠️ 'save_hybrid=True' will cause incorrect mAP.

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:10<00:00,  1.20it/s]


                   all        782        960          1      0.968      0.978      0.978
Speed: 1.6ms preprocess, 1.3ms inference, 0.0ms loss, 2.6ms postprocess per image
Saving /data/lizhi/yolo/validation_results_yolo11_test/metrics/predictions.json...
Results saved to /data/lizhi/yolo/validation_results_yolo11_test/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 6.5ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 6.4ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 9.2ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 8.6ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 8.6ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detections), 8.7ms
image 

In [ ]:
from pathlib import Path
from collections import defaultdict
import pandas as pd

def analyze_dataset(image_root: str, label_root: str):
    # 定义路径
    image_base = Path(image_root)
    label_base = Path(label_root)
    
    # 定义厂商前缀
    vendors = ["Kavo", "Meyer", "Newtown"]
    splits = ["train", "val", "test"]
    
    # 统计数据结构
    image_stats = defaultdict(lambda: defaultdict(int))
    label_stats = defaultdict(lambda: defaultdict(set))  # 使用集合去重
    
    # 遍历所有数据集分割
    for split in splits:
        # 统计图像数量
        img_dir = image_base / split
        for img_file in img_dir.glob("*.*"):
            for vendor in vendors:
                if img_file.name.startswith(vendor):
                    image_stats[split][vendor] += 1
                    break
        
        # 统计标注数量
        label_dir = label_base / split
        for label_file in label_dir.glob("*.txt"):
            # 匹配对应厂商
            vendor = next((v for v in vendors if label_file.name.startswith(v)), None)
            if not vendor:
                continue
            
            # 读取并处理标注
            with open(label_file, 'r') as f:
                for line in f:
                    # 转换为元组去重（可根据需要调整去重粒度）
                    ann = tuple(map(float, line.strip().split()))
                    if len(ann) >= 5:  # 确保有效标注
                        # 使用完整标注信息去重
                        label_stats[vendor][split].add(ann)
    
    # 转换统计结果为表格
    # 图像数量表
    image_table = pd.DataFrame.from_dict(
        {(split, vendor): count 
         for split in image_stats
         for vendor, count in image_stats[split].items()},
        orient='index', 
        columns=['Image Count']
    ).unstack(level=0).fillna(0)
    
    # 标注数量表（去重后）
    label_table = pd.DataFrame.from_dict(
        {(vendor, split): len(anns)
         for vendor in label_stats
         for split, anns in label_stats[vendor].items()},
        orient='index',
        columns=['Unique Annotations']
    ).unstack(level=0).fillna(0)
    
    return image_table.astype(int), label_table.astype(int)

if __name__ == "__main__":
    image_dir = "/data/lizhi/yolo/yolo_dataset/images"
    label_dir = "/data/lizhi/yolo/yolo_dataset/labels"
    
    img_table, ann_table = analyze_dataset(image_dir, label_dir)


In [11]:
    
print("=== 图像数量统计 ===")
print(img_table.to_markdown())

print("\n=== 标注数量统计（去重后） ===")
print(ann_table.to_markdown())

=== 图像数量统计 ===
|                                       |    0 |
|:--------------------------------------|-----:|
| ('Image Count', ('train', 'Meyer'))   | 1894 |
| ('Image Count', ('train', 'Newtown')) |  439 |
| ('Image Count', ('train', 'Kavo'))    |  497 |
| ('Image Count', ('val', 'Meyer'))     |  322 |
| ('Image Count', ('val', 'Newtown'))   |  193 |
| ('Image Count', ('val', 'Kavo'))      |  160 |
| ('Image Count', ('test', 'Meyer'))    |  544 |
| ('Image Count', ('test', 'Kavo'))     |  145 |

=== 标注数量统计（去重后） ===
|                                              |   0 |
|:---------------------------------------------|----:|
| ('Unique Annotations', ('Newtown', 'train')) |   8 |
| ('Unique Annotations', ('Newtown', 'val'))   |   3 |
| ('Unique Annotations', ('Meyer', 'train'))   |  48 |
| ('Unique Annotations', ('Meyer', 'val'))     |   8 |
| ('Unique Annotations', ('Meyer', 'test'))    |  14 |
| ('Unique Annotations', ('Kavo', 'train'))    |  15 |
| ('Unique Annotations', ('Kavo', 

### === 标注数量统计（单文件去重） ===


In [7]:
from pathlib import Path
from collections import defaultdict
import pandas as pd

def analyze_annotations(label_root: str):
    # 定义路径和参数
    label_base = Path(label_root)
    vendors = ["Kavo", "Meyer", "Newtown"]
    splits = ["train", "val", "test"]
    
    # 三级统计结构：厂商 -> 数据集分割 -> 标注总数
    stats = defaultdict(lambda: defaultdict(int))
    
    for split in splits:
        split_dir = label_base / split
        if not split_dir.exists():
            continue
            
        for label_file in split_dir.glob("*.txt"):
            # 识别厂商来源
            vendor = next((v for v in vendors if label_file.name.startswith(v)), None)
            if not vendor:
                continue
                
            # 单文件去重逻辑
            unique_anns = set()
            with open(label_file, 'r') as f:
                for line in f:
                    # 标准化处理标注行
                    parts = line.strip().split()
                    if len(parts) < 5:  # 跳过无效行
                        continue
                        
                    # 转换为标准化元组（保留必要精度）
                    ann = (
                        int(parts[0]),        # 类别ID
                        round(float(parts[1]), 4),  # x_center
                        round(float(parts[2]), 4),  # y_center
                        round(float(parts[3]), 4),  # width
                        round(float(parts[4]), 4)   # height
                    )
                    unique_anns.add(ann)
            
            # 累加当前文件的唯一标注数
            stats[vendor][split] += len(unique_anns)
    
    # 转换为表格
    df = pd.DataFrame.from_dict(
        {(vendor, split): count 
         for vendor in stats
         for split, count in stats[vendor].items()},
        orient='index',
        columns=['Unique Annotations']
    ).unstack(level=0).fillna(0)
    
    return df.astype(int)

if __name__ == "__main__":
    label_dir = "/data/lizhi/yolo/yolo_dataset/labels"
    annotation_stats = analyze_annotations(label_dir)
    
    print("=== 标注数量统计（单文件去重） ===")
    print(annotation_stats.to_markdown())

=== 标注数量统计（单文件去重） ===
|                                              |    0 |
|:---------------------------------------------|-----:|
| ('Unique Annotations', ('Newtown', 'train')) |  439 |
| ('Unique Annotations', ('Newtown', 'val'))   |  193 |
| ('Unique Annotations', ('Meyer', 'train'))   | 1960 |
| ('Unique Annotations', ('Meyer', 'val'))     |  322 |
| ('Unique Annotations', ('Meyer', 'test'))    |  615 |
| ('Unique Annotations', ('Kavo', 'train'))    |  633 |
| ('Unique Annotations', ('Kavo', 'val'))      |  341 |
| ('Unique Annotations', ('Kavo', 'test'))     |  252 |


In [4]:
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt

def validate_model(
    model_path: str,
    data_yaml: str,
    test_images_dir: str,
    output_subdir: Path,
    batch_size: int = 64,
    device: str = "0",
    conf_thres: float = 0.2,
    iou_thres: float = 0.6
):
    """验证单个模型并保存结果到指定目录"""
    output_subdir.mkdir(parents=True, exist_ok=True)
    
    """测试单个模型并保存结果"""
    if "rtdetr" in str(model_path):
        model = RTDETR(model_path)
    if "world" in str(model_path):
        model = YOLOWorld(model_path)

    # 加载模型
    model = YOLO(model_path)
    
    # 第一阶段：指标验证
    val_metrics = model.val(
        data=data_yaml,
        split='test',
        batch=batch_size,
        device=device,
        conf=conf_thres,
        iou=iou_thres,
        save_json=True,
        save_hybrid=False,
        plots=True,
        project=str(output_subdir),
        name='metrics'
    )
    
    # 第二阶段：可视化预测
    predictions = model.predict(
        source=test_images_dir,
        conf=conf_thres,
        save=True,
        save_txt=True,
        save_conf=True,
        save_crop=True,
        show_labels=True,
        show_conf=True,
        show_boxes=True,
        project=str(output_subdir),
        name='visualization',
        exist_ok=True
    )
    
    # 第三阶段：生成附加可视化（如混淆矩阵）
    confusion_matrix_path = output_subdir / 'metrics' / 'confusion_matrix.png'
    if confusion_matrix_path.exists():
        plt.figure(figsize=(10, 7))
        plt.imshow(plt.imread(confusion_matrix_path))
        plt.title('Confusion Matrix')
        plt.axis('off')
        plt.savefig(output_subdir / 'confusion_matrix_visualized.png', bbox_inches='tight')
        plt.close()
    
    print(f"Validation complete for: {model_path}")

def test_all_models(
    model_root_dir: str,
    data_yaml: str,
    test_images_dir: str,
    output_root: str = "/data/lizhi/yolo/validation_results"
):
    """遍历所有模型并执行验证"""
    model_root = Path(model_root_dir)
    output_root_path = Path(output_root)
    
    for model_dir in model_root.iterdir():
        for model_dir_ in model_dir.iterdir():
            if model_dir_.is_dir():
                best_pt_path = model_dir / model_dir_ / "weights" / "best.pt"
                output_subdir = model_dir / "test_results"
            
                if best_pt_path.exists():
                    print(f"Processing model: {model_dir.name}")
                    validate_model(
                        model_path=str(best_pt_path),
                        data_yaml=data_yaml,
                        test_images_dir=test_images_dir,
                        output_subdir=output_subdir,
                        batch_size=64,
                        device="0",
                        conf_thres=0.26,
                        iou_thres=0.6
                    )
                else:
                    print(f"Warning: No best.pt found in {model_dir}")

# 执行测试
test_all_models(
    model_root_dir="/data/lizhi/yolo/yolo_train_results",
    data_yaml="/data/lizhi/yolo/yolo_dataset/dataset.yaml",
    test_images_dir="/data/lizhi/yolo/yolo_dataset/images/test"
)

Processing model: yolov8m
WARNING ⚠️ 'save_hybrid=True' will append ground truth to predictions for autolabelling.
WARNING ⚠️ 'save_hybrid=True' will cause incorrect mAP.

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:08<00:00,  1.60it/s]


                   all        782        960          1      0.968      0.981      0.981
Speed: 1.0ms preprocess, 4.4ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving /data/lizhi/yolo/yolo_train_results/yolov8m/test_results/metrics/predictions.json...
Results saved to /data/lizhi/yolo/yolo_train_results/yolov8m/test_results/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 5.6ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 5.6ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 5.6ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 5.6ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 5.6ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detecti

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:09<00:00,  1.42it/s]


                   all        782        960          1      0.968       0.98      0.979
Speed: 0.9ms preprocess, 6.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving /data/lizhi/yolo/yolo_train_results/yolo12m/test_results/metrics/predictions.json...
Results saved to /data/lizhi/yolo/yolo_train_results/yolo12m/test_results/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 10.8ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 10.8ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 10.8ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 10.9ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 11.2ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no de

...

                   all        782        960      0.691      0.268       0.47      0.171
Speed: 1.2ms preprocess, 2.5ms inference, 0.0ms loss, 0.1ms postprocess per image
Saving /data/lizhi/yolo/yolo_train_results/yolov10s/test_results/metrics/predictions.json...
Results saved to /data/lizhi/yolo/yolo_train_results/yolov10s/test_results/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 7.9ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 7.9ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 7.9ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 8.4ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 8.4ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detec

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:12<00:00,  1.02it/s]


                   all        782        960          1      0.968       0.98       0.98
Speed: 1.3ms preprocess, 10.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving /data/lizhi/yolo/yolo_train_results/yolov5x/test_results/metrics/predictions.json...
Results saved to /data/lizhi/yolo/yolo_train_results/yolov5x/test_results/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 12.6ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 12.4ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 12.2ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 12.1ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 12.1ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no d

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:05<00:00,  2.37it/s]


                   all        782        960          1      0.968       0.98       0.98
Speed: 1.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving /data/lizhi/yolo/yolo_train_results/yolov5n/test_results/metrics/predictions.json...
Results saved to /data/lizhi/yolo/yolo_train_results/yolov5n/test_results/metrics

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 4.0ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 3.9ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 3.9ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 3.9ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 3.9ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detecti

In [6]:
import os
import zipfile
from pathlib import Path

def zip_model_files(model_dir: Path, output_zip: Path):
    """压缩模型文件（排除weights目录和test_results/metrics目录）"""
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(model_dir):
            # 排除权重目录和test_results/metrics目录
            dirs[:] = [d for d in dirs if d not in ["weights", "metrics"]]
            
            # 构造相对路径
            rel_path = root.replace(str(model_dir), "").lstrip('/')
            
            # 进一步排除test_results/metrics路径结构
            if "test_results/metrics" in rel_path:
                continue
                
            for file in files:
                file_path = Path(root) / file
                zip_path = Path(rel_path) / file
                zipf.write(file_path, zip_path)

def pack_all_models(
    model_root_dir: str = "/data/lizhi/yolo/yolo_train_results",
    output_root: str = "/data/lizhi/yolo/packed_results"
):
    """遍历所有模型并压缩（排除指定目录）"""
    model_root = Path(model_root_dir)
    output_root_path = Path(output_root)
    output_root_path.mkdir(parents=True, exist_ok=True)
    
    for model_dir in model_root.iterdir():
        if model_dir.is_dir():
            model_name = model_dir.name
            output_zip = output_root_path / f"{model_name}.zip"
            
            print(f"Started packing {model_name}...")
            
            try:
                zip_model_files(model_dir, output_zip)
                print(f"✅ Done: {output_zip}")
            except Exception as e:
                print(f"❌ Failed: {model_name} → {str(e)}")

# 执行打包
pack_all_models()

Started packing yolov8m...
✅ Done: /data/lizhi/yolo/packed_results/yolov8m.zip
Started packing yolov8x-worldv2...
✅ Done: /data/lizhi/yolo/packed_results/yolov8x-worldv2.zip
Started packing yolo12m...
✅ Done: /data/lizhi/yolo/packed_results/yolo12m.zip
Started packing yolov10n...
✅ Done: /data/lizhi/yolo/packed_results/yolov10n.zip
Started packing yolov10m...
✅ Done: /data/lizhi/yolo/packed_results/yolov10m.zip
Started packing yolov8x...
✅ Done: /data/lizhi/yolo/packed_results/yolov8x.zip
Started packing tooth_defect_nano...
✅ Done: /data/lizhi/yolo/packed_results/tooth_defect_nano.zip
Started packing rtdetr-l...
✅ Done: /data/lizhi/yolo/packed_results/rtdetr-l.zip
Started packing yolo11n...
✅ Done: /data/lizhi/yolo/packed_results/yolo11n.zip
Started packing yolov5s...
✅ Done: /data/lizhi/yolo/packed_results/yolov5s.zip
Started packing yolov5l...
✅ Done: /data/lizhi/yolo/packed_results/yolov5l.zip
Started packing yolov10b...
✅ Done: /data/lizhi/yolo/packed_results/yolov10b.zip
Started 

In [14]:
import os
import yaml
from ultralytics import YOLO
from tqdm import tqdm

def main():
    # 配置路径
    models_root = "/data/lizhi/yolo/yolo_train_results"
    dataset_yaml = "/data/lizhi/yolo/yolo_dataset/dataset.yaml"
    output_root = "/data/lizhi/yolo/yolo_test_results"
    
    # 创建输出目录
    os.makedirs(output_root, exist_ok=True)
    
    # 加载数据配置文件
    with open(dataset_yaml, "r") as f:
        data_config = yaml.safe_load(f)
    
    # 遍历所有模型
    for model_dir in os.listdir(models_root):
        model_path = os.path.join(models_root, model_dir)
        if not os.path.isdir(model_path):
            continue
        
        # 查找 best.pt 权重路径
        best_pt_path = ""
        for root, dirs, files in os.walk(model_path):
            for file in files:
                if file == "best.pt":
                    best_pt_path = os.path.join(root, file)
                    break
            if best_pt_path:
                break
        
        if not best_pt_path:
            print(f"Warning: No best.pt found for model {model_dir}")
            continue
        
        # 创建模型输出目录
        model_output_dir = os.path.join(output_root, model_dir)
        os.makedirs(model_output_dir, exist_ok=True)
        
        # 加载模型
        model = YOLO(best_pt_path)
        
        # 1. 进行测试集推理并保存可视化结果
        predict_output = os.path.join(model_output_dir, "visualizations")
        model.predict(
            source=os.path.join(data_config["path"], data_config["test"]),  # 使用 YAML 中的测试集路径
            save=True,
            save_txt=True,
            project=model_output_dir,
            name="predictions",
            exist_ok=True
        )
        
        # 2. 计算指标并保存
        results = model.val(data=dataset_yaml, split="test")  # 直接指定 YAML 文件路径
        metrics_path = os.path.join(model_output_dir, "metrics.txt")
        with open(metrics_path, "w") as f:
            f.write(str(results))
        
        # 3. 拷贝预测结果到统一目录
        # src_predictions = os.path.join(model_output_dir, "predictions", "test")
        # dst_predictions = os.path.join(model_output_dir, "predictions")
        # os.makedirs(dst_predictions, exist_ok=True)
        # for file in os.listdir(src_predictions):
        #     shutil.copy(
        #         os.path.join(src_predictions, file),
        #         dst_predictions
        #     )
        
        print(f"Processed model: {model_dir}")

if __name__ == "__main__":
    main()


image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 12.7ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 12.5ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 12.8ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 12.8ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 12.7ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detections), 12.7ms
image 7/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice104.png: 640x640 (no detections), 12.6ms
image 8/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice105.png: 640x640 (no detections), 12.7ms
image 9/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice106.png: 640x640 (no detections)

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:09<00:00,  5.20it/s]


                   all        782        960      0.773      0.375      0.487      0.149
Speed: 0.3ms preprocess, 5.3ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/val3
Processed model: yolov8m

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 16.9ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 17.2ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 14.5ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 17.1ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 17.0ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detections), 17.3ms
image 7/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice104.png: 640x640 (no detec

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:12<00:00,  4.00it/s]


                   all        782        960       0.76      0.389      0.516      0.144
Speed: 0.3ms preprocess, 9.0ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val4
Processed model: yolo12m

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 6.9ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 1 tooth_defect, 6.1ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 6.5ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 6.3ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 1 tooth_defect, 6.6ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 1 tooth_defect, 6.1ms
image 7/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice104.png: 640x640 1 tooth_defect, 6.

...

                   all        782        960      0.635      0.302       0.37      0.117
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val22
Processed model: yolov10s

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 12.2ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 12.1ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 12.1ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 12.2ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 12.2ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detections), 12.2ms
image 7/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice104.png: 640x640 (no det

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:17<00:00,  2.83it/s]


                   all        782        960      0.624      0.337      0.435      0.136
Speed: 0.5ms preprocess, 13.8ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/val23
Processed model: yolov5x

image 1/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice098.png: 640x640 (no detections), 6.2ms
image 2/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice099.png: 640x640 (no detections), 6.2ms
image 3/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice100.png: 640x640 (no detections), 6.2ms
image 4/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice101.png: 640x640 (no detections), 6.3ms
image 5/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice102.png: 640x640 (no detections), 6.2ms
image 6/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice103.png: 640x640 (no detections), 6.3ms
image 7/782 /data/lizhi/yolo/yolo_dataset/images/test/Kavo_33.nii_slice104.png: 640x640 (no detection

val: Scanning /data/lizhi/yolo/yolo_dataset/labels/test.cache... 782 images, 0 backgrounds, 0 corrupt: 100%|██████████| 782/782 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 49/49 [00:05<00:00,  9.32it/s]


                   all        782        960      0.655      0.362      0.453      0.125
Speed: 0.3ms preprocess, 1.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val24
Processed model: yolov5n


In [30]:
import pandas as pd
import os
import ast

def aggregate_metrics(input_dir, output_file):
    metrics_data = []
    
    for model_dir in os.listdir(input_dir):
        model_path = os.path.join(input_dir, model_dir)
        if not os.path.isdir(model_path):
            continue
        
        metrics_file = os.path.join(model_path, "metrics.txt")
        if not os.path.exists(metrics_file):
            continue
        
        with open(metrics_file, "r") as f:
            lines = f.readlines()
            metrics = {}
            results_dict_line = None
            
            # 找到 results_dict 行
            for line in lines:
                line = line.strip()
                if line.startswith("results_dict:"):
                    results_dict_line = line
                    break
            
            if not results_dict_line:
                print(f"Warning: No results_dict found in {metrics_file}")
                continue
            
            try:
                # 提取字典部分并清理
                dict_str = results_dict_line.split(": ", 1)[1].strip()
                dict_str = dict_str.replace("'", '"')  # 单引号转双引号
                
                # 确保以 } 结尾
                if not dict_str.endswith("}"):
                    dict_str = dict_str.rstrip("},") + "}"
                
                # 解析字典
                results_dict = ast.literal_eval(dict_str)
                metrics.update(results_dict)
            
            except Exception as e:
                print(f"Error parsing {metrics_file}: {str(e)}")
                continue
            
            # 提取目标指标
            metrics_entry = {
                "Model": model_dir,
                "mAP@0.5": metrics.get("metrics/mAP50(B)", "N/A"),
                "mAP@0.5-0.95": metrics.get("metrics/mAP50-95(B)", "N/A"),
                "Precision": metrics.get("metrics/precision(B)", "N/A"),
                "Recall": metrics.get("metrics/recall(B)", "N/A")
            }
            
            metrics_data.append(metrics_entry)
    
    df = pd.DataFrame(metrics_data)
    df.to_csv(output_file, index=False)

# 使用示例
aggregate_metrics(
    input_dir="/data/lizhi/yolo/yolo_test_results",
    output_file="/data/lizhi/yolo/yolo_test_results/metrics_summary.csv"
)

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def plot_metrics(csv_path, output_dir):
    # 读取数据
    df = pd.read_csv(csv_path)
    
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 指标列表
    metrics = ['mAP@0.5', 'mAP@0.5-0.95', 'Precision', 'Recall']
    
    # Seaborn 美化设置
    sns.set(style="whitegrid", font_scale=1.2)
    
    for metric in metrics:
        # 按指标排序（降序）
        sorted_df = df.sort_values(by=metric, ascending=False)
        
        # 创建画布和坐标轴
        plt.figure(figsize=(14, 7))
        ax = sns.barplot(
            x='Model',
            y=metric,
            data=sorted_df,
            palette='viridis'
        )
        
        # 旋转 x 轴标签
        plt.xticks(rotation=45, ha='right', fontsize=10)
        
        # 添加数值标签
        for p in ax.patches:
            value = f"{p.get_height():.3f}"
            ax.text(p.get_x() + p.get_width()/2., p.get_height(),
                    value,
                    ha='center',
                    va='bottom',
                    fontsize=9)
        
        # 设置标题和标签
        plt.title(f"{metric} Comparison Across Models", fontsize=16, pad=20)
        plt.xlabel("Model", fontsize=12)
        plt.ylabel(f"{metric} Value", fontsize=12)
        
        # 调整布局并保存
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"{metric}_comparison.png"), dpi=300, bbox_inches='tight')
        plt.close()

# 使用示例
plot_metrics(
    csv_path="/data/lizhi/yolo/yolo_test_results/metrics_summary.csv",
    output_dir="/data/lizhi/yolo/yolo_test_results/plots"
)

In [24]:
import shutil

def create_comparison_dir(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for model in os.listdir(input_dir):
        src_dir = os.path.join(input_dir, model, "visualizations")
        dst_dir = os.path.join(output_dir, model)
        shutil.copytree(src_dir, dst_dir, dirs_exist_ok=True)

create_comparison_dir(
    input_dir="/data/lizhi/yolo/yolo_test_results",
    output_dir="/data/lizhi/yolo/yolo_test_results/visualizations_all"
)

FileNotFoundError: [Errno 2] No such file or directory: '/data/lizhi/yolo/yolo_test_results/yolov8m/visualizations'

# 4分类

In [19]:
import os
import random
import shutil
from tqdm import tqdm

def split_dataset(input_dir, output_root, splits={"train": 0.7, "val": 0.15, "test": 0.15}):
    # 创建输出目录结构
    for split in splits.keys():
        for cls in ["class0", "class1", "class2", "class3"]:
            os.makedirs(os.path.join(output_root, split, cls), exist_ok=True)
    
    # 遍历每个类别
    for cls in ["class0", "class1", "class2", "class3"]:
        class_dir = os.path.join(input_dir, cls)
        images = [f for f in os.listdir(class_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        random.shuffle(images)
        total = len(images)
        
        # 计算划分数量
        train_size = int(total * splits["train"])
        val_size = int(total * splits["val"])
        
        # 分配数据
        train_imgs = images[:train_size]
        val_imgs = images[train_size:train_size+val_size]
        test_imgs = images[train_size+val_size:]
        
        # 复制文件
        for img in tqdm(train_imgs, desc=f"Copying {cls} to train"):
            shutil.copy(
                os.path.join(class_dir, img),
                os.path.join(output_root, "train", cls, img)
            )
        
        for img in tqdm(val_imgs, desc=f"Copying {cls} to val"):
            shutil.copy(
                os.path.join(class_dir, img),
                os.path.join(output_root, "val", cls, img)
            )
        
        for img in tqdm(test_imgs, desc=f"Copying {cls} to test"):
            shutil.copy(
                os.path.join(class_dir, img),
                os.path.join(output_root, "test", cls, img)
            )

if __name__ == "__main__":
    input_dir = "/data/lizhi/yolo/dataset/yolo_dataset_class"
    output_root = "/data/lizhi/yolo/dataset/yolo_dataset_class_split"
    split_dataset(input_dir, output_root)

Copying class3 to train: 100%|██████████| 2/2 [00:00<00:00, 962.88it/s]
Copying class3 to val: 0it [00:00, ?it/s]
Copying class3 to test: 100%|██████████| 2/2 [00:00<00:00, 3260.24it/s]


# 分类模型训练

In [9]:
from ultralytics import YOLO

def main():
    # 加载最佳模型
    model = YOLO("/data/lizhi/yolo/classification_runs/tooth_defect_classifier8/weights/best.pt")
    
    # 验证测试集（确保 data.yaml 包含 test 路径）
    results = model.val(
        data="/data/lizhi/yolo/dataset/yolo_dataset_class_split/dataset.yaml",  # 包含 test 路径的 YAML 文件
        split="test"  # 明确指定测试集
    )
    
    # 打印评估结果
    print(results)
    print(f"Top-1 Accuracy: {results.top1:.4f}")
    print(f"Top-5 Accuracy: {results.top5:.4f}")
    print(f"Inference Speed: {results.speed['inference']:.1f} ms per image")
    print(f"Task: {results.task}")

if __name__ == "__main__":
    main()

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)


YOLOv8n-cls summary (fused): 30 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs


test: Scanning /data/lizhi/yolo/dataset/yolo_dataset_class_split/test... 57 images, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:00<00:00,  9.63it/s]


                   all          1          1
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/classify/val4
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fe864d12570>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: PosixPath('runs/classify/val4')
speed: {'preprocess': 0.4031558775367509, 'inference': 3.0194381405109247, 'loss': 0.005519772234389134, 'postprocess': 0.06982564911850841}
task: 'classify'
top1: 1.0
top5: 1.0
Top-1 Accuracy: 1.0000
Top-5 Accuracy: 1.0000
Inference Speed: 3.0 ms per image
Task: classify


In [7]:
from ultralytics import YOLO
import os

def main():
    # 加载最佳模型
    model = YOLO("/data/lizhi/yolo/classification_runs/tooth_defect_classifier8/weights/best.pt")
    
    # 测试集路径
    test_dir = "/data/lizhi/yolo/dataset/yolo_dataset_class_split/test"
    
    # 遍历测试集所有图像并预测
    correct = 0
    total = 0
    class_counts = [0] * model.names  # 统计每个类别的正确预测数
    
    for class_folder in os.listdir(test_dir):
        class_path = os.path.join(test_dir, class_folder)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            results = model.predict(source=img_path, verbose=False)
            
            # 获取预测结果
            pred_class = results[0].probs.top1  # 最可能的类别索引
            true_class = int(class_folder)  # 假设文件夹名是类别索引（如 '0', '1', '2', '3'）
            
            if pred_class == true_class:
                correct += 1
                class_counts[pred_class] += 1
            total += 1
    
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")
    print("Class-wise Accuracy:")
    for i, count in enumerate(class_counts):
        print(f"Class {i}: {count}/{total} ({count/total:.4f})")

if __name__ == "__main__":
    main()

TypeError: can't multiply sequence by non-int of type 'dict'

In [4]:
from ultralytics import YOLO

def main():
    # 加载预训练分类模型（如 YOLOv11 的分类版本）
    model = YOLO("yolo11n-cls.pt")  # 确保文件存在
    
    # 适配您的数据集参数
    model.model.nc = 4  # 类别数量
    model.model.names = ["class0", "class1", "class2", "class3"]  # 类别名称
    
    # 训练配置
    results = model.train(
        data="/data/lizhi/yolo/dataset/yolo_dataset_class_split/dataset.yaml",
        epochs=150,
        imgsz=224,  # 推荐与预训练模型尺寸一致（如 224）
        project="classification_runs",
        name="tooth_defect_classifier",
        pretrained=True,  # 保持预训练权重
        freeze=8,         # 冻结主干网络（仅训练分类头）
        lr0=0.001         # 调整学习率
    )

if __name__ == "__main__":
    main()

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/data/lizhi/yolo/dataset/yolo_dataset_class_split/dataset.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=classification_runs, name=tooth_defect_classifier2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=8, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

KeyboardInterrupt: 

In [21]:
from ultralytics import YOLO

model = YOLO("yolo11n.yaml")  # 使用检测模型配置
model.train(
    data="/data/lizhi/yolo/dataset/yolo_dataset_detect/dataset.yaml",
    epochs=300,
    imgsz=640,
    project="yolo_train_results_4class",
    name="tooth_defect_detection"
)

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=/data/lizhi/yolo/dataset/yolo_dataset_detect/dataset.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolo_train_results_4class, name=tooth_defect_detection2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

train: Scanning /data/lizhi/yolo/dataset/yolo_dataset_detect/labels/train.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]
val: Scanning /data/lizhi/yolo/dataset/yolo_dataset_detect/labels/val.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


Plotting labels to yolo_train_results_4class/tooth_defect_detection2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_train_results_4class/tooth_defect_detection2
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/300      2.76G      5.297      10.39      4.222         15        640: 100%|██████████| 13/13 [00:03<00:00,  3.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  6.45it/s]

                   all        114        121          0          0          0          0


...

                   all        114        121      0.587      0.684      0.699      0.446



300 epochs completed in 0.224 hours.
Optimizer stripped from yolo_train_results_4class/tooth_defect_detection2/weights/last.pt, 5.5MB
Optimizer stripped from yolo_train_results_4class/tooth_defect_detection2/weights/best.pt, 5.5MB

Validating yolo_train_results_4class/tooth_defect_detection2/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.36it/s]


                   all        114        121      0.566      0.694      0.688      0.448
                class0         91         91      0.704       0.89      0.889      0.607
                class1         26         26      0.538      0.885      0.836      0.577
                class2          2          2      0.669        0.5      0.499        0.3
                class3          2          2      0.352        0.5      0.529      0.309
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to yolo_train_results_4class/tooth_defect_detection2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6ed1ed6b10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

# 额外的检测结果

In [24]:
from ultralytics import YOLO
import os

def main():
    # 模型路径（训练好的单类别检测模型）
    model_path = "/data/lizhi/yolo/yolo_train_results/yolov5n/tooth_defect_v5n/weights/best.pt"
    
    # 测试集路径（新数据集的测试集）
    test_dir = "/data/lizhi/yolo/dataset/yolo_dataset_detect/images/test"
    
    # 输出路径（保存检测结果）
    output_dir = "/data/lizhi/yolo/additional_detect_results/yolov5n"
    os.makedirs(output_dir, exist_ok=True)  # 确保目录存在

    # 加载模型
    model = YOLO(model_path)
    
    # 进行检测并保存结果
    results = model(
        source=test_dir,
        save=True,               # 保存检测后的图像
        project=output_dir,      # 输出父目录
        name="detections",       # 子目录名称（如 yolo11n/detections）
        save_txt=True,           # 保存文本格式的预测结果（包含坐标和置信度）
        save_conf=True,          # 在文本文件中保存置信度
        exist_ok=True            # 允许覆盖已有文件
    )

    print(f"检测结果已保存到: {os.path.join(output_dir, 'detections')}")

if __name__ == "__main__":
    main()


image 1/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0056.png: 640x640 1 tooth_defect, 8.7ms
image 2/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0059.png: 640x640 1 tooth_defect, 6.9ms
image 3/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0060.png: 640x640 1 tooth_defect, 9.7ms
image 4/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0061.png: 640x640 (no detections), 6.3ms
image 5/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0062.png: 640x640 1 tooth_defect, 6.5ms
image 6/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0063.png: 640x640 (no detections), 6.7ms
image 7/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0064.png: 640x640 1 tooth_defect, 6.0ms
image 8/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0066.png: 640x640 1 tooth_defect, 8.4ms
image 9/119 /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_00

In [26]:
from ultralytics import YOLO
import os

def main():
    # 模型路径
    model_path = "/data/lizhi/yolo/yolo_train_results/yolov5n/tooth_defect_v5n/weights/best.pt"
    
    # 临时数据集配置路径
    temp_yaml = "/data/lizhi/yolo/dataset/yolo_dataset_detect/dataset.yaml"  # 确保文件存在
    
    # 加载模型
    model = YOLO(model_path)
    
    # 使用临时配置进行评估
    results = model.val(
        data=temp_yaml,  # 临时配置文件路径
        split="test",    # 评估测试集（合并后的所有数据）
        save_txt=False,  # 不保存结果，仅计算指标
        save_conf=False
    )
    
    # 打印整体检测指标
    print("整体检测指标：")
    print(f"mAP50-95: {results.box.map:.4f}")
    print(f"mAP50:    {results.box.map50:.4f}")

if __name__ == "__main__":
    main()

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
YOLOv5n summary (fused): 84 layers, 2,503,139 parameters, 0 gradients, 7.1 GFLOPs


val: Scanning /data/lizhi/yolo/dataset/yolo_dataset_detect/labels/test.cache... 119 images, 0 backgrounds, 26 corrupt: 100%|██████████| 119/119 [00:00<?, ?it/s]

val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0060.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0062.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0063.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0068.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0069.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yo


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.49it/s]


                   all         93         93        0.8      0.515      0.589      0.274
Speed: 0.5ms preprocess, 2.0ms inference, 0.1ms loss, 5.7ms postprocess per image
Results saved to runs/detect/val32
整体检测指标：
mAP50-95: 0.2742
mAP50:    0.5888


In [ ]:
# 模型路径
model_path = "/data/lizhi/yolo/yolo_train_results/yolo11n/tooth_defect_12n/weights/best.pt"

# 临时数据集配置路径
temp_yaml = "/data/lizhi/yolo/dataset/yolo_dataset_detect/dataset.yaml"  # 确保文件存在

# 加载模型
model = YOLO(model_path)

# 使用临时配置进行评估
results = model.val(
    data=temp_yaml,  # 临时配置文件路径
    split="test",    # 评估测试集（合并后的所有数据）
    save_txt=False,  # 不保存结果，仅计算指标
    save_conf=False
)

# 打印整体检测指标
print("整体检测指标：")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"mAP50:    {results.box.map50:.4f}")

Ultralytics 8.3.94 🚀 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A6000, 48670MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /data/lizhi/yolo/dataset/yolo_dataset_detect/labels/test.cache... 119 images, 0 backgrounds, 26 corrupt: 100%|██████████| 119/119 [00:00<?, ?it/s]

val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0060.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0062.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0063.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0068.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yolo/dataset/yolo_dataset_detect/images/test/slice_0069.png: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING ⚠️ /data/lizhi/yo


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:01<00:00,  3.24it/s]


                   all         93         93      0.782      0.484      0.546      0.313
Speed: 0.3ms preprocess, 1.7ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/val29
整体检测指标：
mAP50-95: 0.3133
mAP50:    0.5461


AttributeError: 'Metric' object has no attribute 'precision'. See valid attributes below.

    Class for computing evaluation metrics for YOLOv8 model.

    Attributes:
        p (list): Precision for each class. Shape: (nc,).
        r (list): Recall for each class. Shape: (nc,).
        f1 (list): F1 score for each class. Shape: (nc,).
        all_ap (list): AP scores for all classes and all IoU thresholds. Shape: (nc, 10).
        ap_class_index (list): Index of class for each AP score. Shape: (nc,).
        nc (int): Number of classes.

    Methods:
        ap50(): AP at IoU threshold of 0.5 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        ap(): AP at IoU thresholds from 0.5 to 0.95 for all classes. Returns: List of AP scores. Shape: (nc,) or [].
        mp(): Mean precision of all classes. Returns: Float.
        mr(): Mean recall of all classes. Returns: Float.
        map50(): Mean AP at IoU threshold of 0.5 for all classes. Returns: Float.
        map75(

In [17]:
# 打印整体检测指标
print("整体检测指标：")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"mAP50:    {results.box.map50:.4f}")
print(f"Precision: {results.box.mean_results:.4f}")
print(f"Recall:    {results.box.class_result:.4f}")
print(f"Recall:    {results.box.ap50:.4f}")
print(f"Recall:    {results.box.ap:.4f}")
print(f"Recall:    {results.box.fitness:.4f}")


整体检测指标：
mAP50-95: 0.3133
mAP50:    0.5461


TypeError: unsupported format string passed to method.__format__